# Census API Data
## Using CensusReporter data to market to target audiences
### Homework for Digital Advertising

In [21]:
import pandas as pd
import json
import requests
import time

df = pd.read_csv( 'msas.csv' )

df.head()

,CBSA,NAME,POPESTIMATE2017
0,10180,"Abilene, TX",170219
1,10420,"Akron, OH",703505
2,10500,"Albany, GA",151434
3,10540,"Albany, OR",125047
4,10580,"Albany-Schenectady-Troy, NY",886188


In [7]:
# get table ids with target information
tableids = [ 'B01001', 'B03002', 'B19001' ]

# format table ids to be recognized by JSON
tableidstring = ','.join(tableids)

In [8]:
# create variables
geo = 0
geoid = []
url_list = []

In [32]:
# create list of URLs for each geoid
for index, row in df.iterrows():
    geo = '31000US' + str(df.CBSA[index])
    url = 'http://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=%s' % (tableidstring, geo)
    url_list.append(url)
    geoid.append(geo)

In [10]:
# use JSON editor to find target age codes within each table
ageids = [ 'B01001007', 'B01001008', 'B01001009', 'B01001010', 'B01001011', 'B01001012',
          'B01001031', 'B01001032', 'B01001033', 'B01001034', 'B01001035', 'B01001036' ]

In [11]:
# create lists for target data
age = []
race = []
income = []

In [12]:
# iterate through each url and return target data
for index in url_list:
    loadedjson = requests.get(index)
    loadedjson.json()
    parsedjson = json.loads(loadedjson.text)
    # remove from current url
    locstr = index[-12:]
    # create temporary df for each parameter
    agedf = pd.DataFrame.from_dict( parsedjson['data'][locstr]['B01001'] )
    racedf = pd.DataFrame.from_dict( parsedjson['data'][locstr]['B03002'] )
    incomedf = pd.DataFrame.from_dict( parsedjson['data'][locstr]['B19001'] )
    # sum age codes
    age_sum = 0
    for agepop in ageids:
        age_sum = agedf['estimate'][agepop] + age_sum
    # append sum to age list
    age.append(age_sum)
    # append hispanic pop to race list
    race.append( racedf['estimate']['B03002012'] )
    # append income pop to income list
    income.append( incomedf['estimate']['B19001011'] + incomedf['estimate']['B19001012'] )
    # use sleep because api overloads
    time.sleep(-time.time()%1)

In [19]:
# create dataframe for final data
fulldf = pd.DataFrame()
fulldf['url'] = url_list
fulldf['geo'] = geoid
fulldf['name'] = df['NAME']

# add new lists to full dataframe
fulldf['age'] = age
fulldf['race'] = race
fulldf['income'] = income

fulldf.head()

,url,geo,name,age,race,income
0,http://api.censusreporter.org/1.0/data/show/la...,31000US10180,"Abilene, TX",47469.0,41291.0,11050.0
1,http://api.censusreporter.org/1.0/data/show/la...,31000US10420,"Akron, OH",163361.0,14258.0,54391.0
2,http://api.censusreporter.org/1.0/data/show/la...,31000US10500,"Albany, GA",34688.0,NaN,9948.0
3,http://api.censusreporter.org/1.0/data/show/la...,31000US10540,"Albany, OR",25334.0,11395.0,9131.0
4,http://api.censusreporter.org/1.0/data/show/la...,31000US10580,"Albany-Schenectady-Troy, NY",211611.0,45781.0,56946.0


In [23]:
# Question 1
# return metro area with most 50k-75k households
fulldf.loc[ fulldf['income'].idxmax() ]

url       http://api.censusreporter.org/1.0/data/show/la...
geo                                            31000US35620
name                  New York-Newark-Jersey City, NY-NJ-PA
age                                             4.80694e+06
race                                             4.9944e+06
income                                          1.03819e+06
Name: 249, dtype: object

In [24]:
max_income = fulldf['income'].max()
print( 'Max Income' )
print( max_income )

Max Income
1038188.0


In [25]:
# Question 2
# return metro area with most people age 18-34
fulldf.loc[ fulldf['age'].idxmax() ]

url       http://api.censusreporter.org/1.0/data/show/la...
geo                                            31000US35620
name                  New York-Newark-Jersey City, NY-NJ-PA
age                                             4.80694e+06
race                                             4.9944e+06
income                                          1.03819e+06
Name: 249, dtype: object

In [26]:
max_age = fulldf['age'].max()
print( 'Max Age' )
print( max_age )

Max Age
4806939.0


In [27]:
# Question 3
# return metro area with largest hispanic population
fulldf.loc[ fulldf['race'].idxmax() ]

url       http://api.censusreporter.org/1.0/data/show/la...
geo                                            31000US31080
name                     Los Angeles-Long Beach-Anaheim, CA
age                                             3.38111e+06
race                                            6.03149e+06
income                                               708153
Name: 210, dtype: object

In [28]:
max_race = fulldf['race'].max()
print( 'Max Race' )
print( max_race )

Max Race
6031492.0


In [29]:
# Question 4
# find metro area with highest average percentage
# create percentages for each parameter
fulldf['age_per'] = fulldf['age'] / max_age * 100
fulldf['race_per'] = fulldf['race'] / max_race * 100
fulldf['income_per'] = fulldf['income'] / max_income * 100

# average all three percentages
fulldf['avg_per'] = ( fulldf['age_per'] + fulldf['race_per'] + fulldf['income_per'] ) / 3

In [30]:
# return highest average percentage
fulldf.loc[ fulldf['avg_per'].idxmax() ]

url           http://api.censusreporter.org/1.0/data/show/la...
geo                                                31000US35620
name                      New York-Newark-Jersey City, NY-NJ-PA
age                                                 4.80694e+06
race                                                 4.9944e+06
income                                              1.03819e+06
age_per                                                     100
race_per                                                82.8053
income_per                                                  100
avg_per                                                 94.2684
Name: 249, dtype: object

In [31]:
max_avg = fulldf['avg_per'].max()
print( 'Max Average' )
print( max_avg )

Max Average
94.26843308421864
